In [ ]:
# Required packages and modules for simulating and visualizing Crazyflie drone behavior
from quad_utils import plotting, Crazyflie as CF, animate_quad
import matplotlib.pyplot as plt  # For plotting drone trajectories and diagnostics
import numpy as np               # For numerical operations
from scipy.linalg import solve_continuous_are  # For solving the continuous-time algebraic Riccati equation (LQR)
from IPython.display import HTML, Image         # For embedding animations and images in the notebook


###Part 1: Setting up Drone Connection
Instructions

Before we begin flying or simulating the Crazyflie drone, we need to establish a connection to the correct drone unit. For this project, we will all be using the same drone, which has the group_number 16.


Update the group_num variable below with the drone's number,  and run the code block to initiate the connection.

If successful, you will start seeing real-time telemetry data printed in the output — including roll, pitch, and yaw (orientation angles from the IMU).


In [ ]:
group_number =

In [ ]:
# This is an example from the Crazyflie Python API.
# See https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/basiclogSync.py

import logging
import time

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)


# Initialize the low-level drivers (don't list the debug drivers)
cflib.crtp.init_drivers(enable_debug_driver=False)
# Scan for Crazyflies and use the first one found
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    lg_stab = LogConfig(name='Stabilizer', period_in_ms=10)
    lg_stab.add_variable('stabilizer.roll', 'float')
    lg_stab.add_variable('stabilizer.pitch', 'float')
    lg_stab.add_variable('stabilizer.yaw', 'float')

    cf = Crazyflie(rw_cache='./cache')
    with SyncCrazyflie(available[0][0], cf=cf) as scf:
        with SyncLogger(scf, lg_stab) as logger:
            endTime = time.time() + 10

            for log_entry in logger:
                timestamp = log_entry[0]
                data = log_entry[1]
                logconf_name = log_entry[2]

                print('[%d][%s]: %s' % (timestamp, logconf_name, data))

                if time.time() > endTime:
                    break

Scanning interfaces for Crazyflies...
Cannot find a Crazyradio Dongle
Crazyflies found:
No Crazyflies found, cannot run example


#Part 2: Programming Drone Movement

###Introduction & Context
In this section, you’ll explore how to write Python code to control a Crazyflie drone's physical movement. We’ll start by initializing a connection to the drone, then:

1. Wait for accurate position estimates from the onboard Kalman filter.

2. Set up the PID controller, which will manage low-level stabilization.

3. Command the drone to take off, hover, and follow a custom trajectory made up of waypoints.

4. Finally, land and stop the drone safely.

This code uses the official Bitcraze Python library for controlling the drone. It's been adapted to let us build high-level movement behaviors by sending position setpoints (i.e. “go to this x, y coordinate”).

You’ll also design a custom helper class, Drone_Controller, which lets you convert high-level movement instructions like "up", "left", etc., into detailed position commands that the drone can follow.

Here are a few helpful functions that are already defined for you:

In [ ]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
# CrazyFlie imports:
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger

## Some helper control functions:
## -----------------------------------------------------------------------------------------

# Determine initial position:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

# Initialize controller:
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')

    wait_for_position_estimator(cf)
    time.sleep(0.1)
    return


Warm-Up: Fill in the Flight Control Functions

✨ Goal
This is a warm-up to help you understand how we use Python to control physical drone behaviors like takeoff, hover, move, and land.

You’ll be working with three flight routines:

1. ascend_and_hover(cf) — makes the drone take off and stay at a fixed height.

2. run_sequence(scf, sequence, setpoint_delay) — sends the drone through a series of waypoints.

3. hover_and_descend(cf) — makes the drone hover in place and then land smoothly.

Each of these functions is written using the Crazyflie command interface:

- cf.commander.send_hover_setpoint(...) controls velocity and height.

- cf.commander.send_position_setpoint(...) commands the drone to move to a specific position.

```
# Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)

# Go to a position -> (x, y, z, 0.0)
cf.commander.send_position_setpoint(position[0],
                                    (position[1]),
                                      0.5,
                                      0.0)
    
```

🧩 Your Task
You’ll be given incomplete versions of each of these functions. Your job is to:

- Use loops to repeat commands over short time intervals.

- Adjust the height or position setpoints to achieve smooth motion.

- Control time between setpoints using time.sleep().

🧠 Hint: Drones don’t respond instantly — they need repeated commands at high frequency to maintain stability!




In [ ]:
# Ascend and hover:
def ascend_and_hover(cf):
    # Ascend -- warmup drone:
    for y in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 50)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(30):
        #TODO: write the command to hover at .5 meters
        ...
        time.sleep(0.1)
    return

# Follow the setpoint sequence trajectory:
def run_sequence(scf, sequence, setpoint_delay):
    cf = scf.cf
    # TODO: write the for loop that loops over positions in the sequence
    for ... in ...:
        print(f'Setting position {(position[0], (position[1]))}')
        # "setpoint delay" is a parameter to give the drone time to reach the set point
        for i in range(setpoint_delay):
            ...
            time.sleep(0.1)

# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    # Hover at 0.5 meters:
    for _ in range(30):
        ...
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

In this section, you’ll put together everything you’ve learned so far — and take full control of how a Crazyflie drone moves through space by designing and executing your own path.

You’re now working with a flexible and modular framework that allows the drone to:

- Interpret a high-level movement plan, written as simple actions like "up" or "left".

- Convert those actions into setpoints that the drone understands — real coordinates in meters.

- Fly the trajectory autonomously, using the run_setpoint_trajectory() function.


🛠 Key Components

####🧠 Drone_Controller:
This class helps bridge the gap between human-readable motion plans and drone-executable coordinates. You now have two ways to create motion:

- convert_traj_to_setpoint(...) — uses a Trajectory policy to generate a sequence of actions from a start and goal. This will be useful when using our QLearning code later

- convert_actions_to_setpoint(...) — lets you directly feed in a list of "up", "right", etc., actions.

Both return a list of 2D position setpoints the drone can fly through.


🧩 Your Task

In this warm-up, you’ll be given a partially written version of these functions. You will:

- Finish the logic to compute and execute motion sequences.

- Choose whether to create your flight plan using a policy (start → goal) or a hand-written action list.

- Make sure your trajectories are converted into the right format — with physical units and smooth transitions.

In [ ]:
def run_setpoint_trajectory(group_number, sequence):
    # This is the main function to enable the drone to follow the trajectory.

    # User inputs:
    #
    # - group_number: (int) the number corresponding to the drone radio settings.
    #
    # - sequence: a series of point locations (float) defined as a numpy array, with each row in the following format:
    #     [x(meters), y(meters)]
    #   Note: the input should be given in drone coordinates (where positive x is forward, and positive y is to the left).
    # Example:
    # sequence = [
    #     [[ 0.          0.        ]
    #      [0.18134891  0.08433607]]
    #

    # Outputs:
    # None.

    setpoint_delay = 3  # Number of 0.1s steps to spend at each setpoint

    # Set the URI the Crazyflie will connect to
    uri = f"radio://0/{group_number}/2M"

    # Initialize Crazyflie radio drivers
    cflib.crtp.init_drivers(enable_debug_driver=False)

    # Connect to the drone and run the control loop
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf

        # Initialize the PID controller and Kalman estimator
        set_PID_controller(cf)

        # TODO: Ascend to safe height before moving
        ...

        # TODO: Run the sequence of setpoints
        ...

        # TODO: Descend and stop all motion
        ...

    print("Done!")
    return

class Drone_Controller:

  def __init__(self, grid_width, rows, cols, group_number):
      self.grid_width = grid_width
      self.rows = rows
      self.cols = cols
      self.position = np.array([0,0])
      self.group_number = group_number

  def discretize_move(self, action, start_point, segments):

    #TODO: initialize move_seq as a zero array... what shape do we need?
    move_seq = ...
    cur = start_point
    move_seq[0] = cur
    increment = ...

    #TODO -- implement our discretizing strategy on a move!
    # HINT: going "up" means positive in the x-value, "left" is positive in the y-value
    #       Think about the Right hand rule!

    if action == "up":
      for i in range(...):
        move_seq[i,0] = ...
        move_seq[i,1] = ...

    elif action == "down":
      for i in range(...):
        move_seq[i,0] = ...
        move_seq[i,1] = ...

    elif action == "left":
      for i in range(...):
        move_seq[i,0] = ...
        move_seq[i,1] = ...

    elif action == "right":
      for i in range(...):
        move_seq[i,0] = ...
        move_seq[i,1] = ...
    print("adding, ", move_seq)
    return move_seq


  def convert_traj_to_setpoint(self, policy, start, goal, segments, drone_init_pos):
      """
      Converts a list of actions (up/down/left/right) into a sequence of setpoints.

      Parameters:
      - policy: policy recovered by QLearning
      - start: grid position of start in the GridWorld, (x,y)
      - goal: grid position of the end in the GridWorld, (x,y)
      - segments: the amount by which we discretize each move
      - drone_init_pos: the literal start point of the drone, usually (0,0)

      """
      # TODO: make a trajectory and find a list of actions using trajectory.step()
      trajectory = Trajectory(policy)
      actions = ...

      #initialize setpoints
      setpoints = np.array(drone_init_pos).reshape((1,2))

      # TODO: loop through the actions in our list, and calculate the next set of
      #       setpoints. Add them to our setpoints using np.concatenate((array 1, array 2), axis = 0)
      # HINT: we need to make sure we start each new setpoint from our last known position.
      #       using the index [-1] gets us to the end of a list...
      for a in actions:

        next_seg = ...
        setpoints = ...

      return setpoints

  def convert_actions_to_setpoint(self, actions, segments, drone_init_pos):
      setpoints = np.array(drone_init_pos).reshape((1,2))

      # TODO: loop through the actions in our list, and calculate the next set of
      #       setpoints. Add them to our setpoints using np.concatenate((array 1, array 2), axis = 0)
      # HINT: we need to make sure we start each new setpoint from our last known position.
      #       using the index [-1] gets us to the end of a list...

      for a in actions:

        next_seg = ...
        setpoints = ...

      return setpoints


  def execute_sequence(self, sequence):
    run_setpoint_trajectory(self.group_number, sequence)
    return


Let's write the code to test our movement protocols with an input list of actions!

In [ ]:
# Initialize our drone with the correct step size and variables
drone = Drone_Controller(...)

# Make a sequence using our movement actions:
actions = ["up", "left", "right", "down"]

input = ["up", "right", "up", "right"]


seq = drone.convert_action_to_setpoint(input, 5, (0,0))

drone.execute_sequence(seq)

🧠 Reflection Questions

1. Why do we break down each action (e.g., "up") into multiple small setpoints instead of sending one big jump?

2. What’s the benefit of separating the trajectory generation (convert_actions_to_setpoint) from execution (execute_sequence)?

3. What might happen if you forget to ascend the drone before running your waypoint sequence?

4. How would you modify discretize_move() to support diagonal or curved motion?

5. When might you want to use a Trajectory policy vs. specifying actions directly?

#Part 3: Encoding the Real-World Setup

The next part of our project is figuring out how best to use our existing Q-Learning and GridWorld environment to represent the maze our drone must solve in real life. This includes correctly figuring out the scale of the maze, where the obstacles are, and where the drone needs to be. Work in groups in the lab to find these values and model the maze.

In [ ]:
# Width and height of the rectangular domain:
width = 0.0  # total width of the cage
height = 0.0 # total height of the cage

rows, cols =

grid_width =

# Obstacles are represented as tuples, where the first element is an np.ndarray containing the center
# and the second element is the radius of the obstacle. For example (np.array([3, 4]), 5).
# This variable is a list of such tuples.
obstacles = []

# The goal is represented in the same way as an obstacle.
goal = ()

# The starting position of the robot.
origin = np.array([])

SyntaxError: invalid syntax (3970833078.py, line 5)

Below is our implementation of the Q-Learning algorithm from yesterday, with most of the code hidden for space. MAKE SURE TO RUN THIS CODE. A lot of the changes / additions we'll write to the code will be in the form of "wrapper" functions, which simply means that we build our new tools to work on top of the existing code that we know works, instead of changing it directly.


In [ ]:
# @title
from typing import Tuple
%pip install celluloid
from celluloid import Camera # getting the camera
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm, Normalize
import numpy as np
import random


### Visualization Tools ###

### Visualization Tools ###

class Move_anim:
    """
    This class provides functionality for visualizing the motion of a trained
    agent in real-time using matplotlib and a camera-like snapshot tool.

    Args:
        ax_obj (matplotlib axis): The axis to draw the animation on.
        cam_obj (camera object): The camera used to capture frames (e.g., celluloid).
        obs (list or np.ndarray): List of obstacle coordinates.
        goal (list or np.ndarray): Goal coordinate.
        bounds (list): Boundary coordinates.
        grid_size (float): Size of each grid cell in the animation.
        T (float): Time to animate a single move.
        invert (bool): Whether to swap row/col in coordinate conversion.
    """

    def __init__(self, ax_obj, cam_obj, obs, goal, bounds, grid_size=1, T=1, invert=False):
        self.move_time = T
        self.grid_size = grid_size
        self.ax = ax_obj
        self.camera = cam_obj
        self.invert = invert
        self._legend_drawn = False

        # Ensure obs and goal are numpy arrays
        self.obs = np.array(obs) if not isinstance(obs, np.ndarray) else obs
        self.goal = np.array(goal) if not isinstance(goal, np.ndarray) else goal
        self.bounds = bounds


    def right(self, x, y):
        "Animate a rightward move"
        for i in range(int(self.move_time * 10)):
            x += self.grid_size / (10 * self.move_time)
            self.ax.scatter(x, y, c='black', edgecolors='white', s=120, marker='o')
            self.show_obs()
            self.camera.snap()
        return x, y


    def left(self, x, y):
        "Animate a leftward move"
        for i in range(int(self.move_time * 10)):
            x -= self.grid_size / (10 * self.move_time)
            self.ax.scatter(x, y, c='black', edgecolors='white', s=120, marker='o')
            self.show_obs()
            self.camera.snap()
        return x, y


    def up(self, x, y):
        "Animate an upward move"
        for i in range(int(self.move_time * 10)):
            y -= self.grid_size / (10 * self.move_time)
            self.ax.scatter(x, y, c='black', edgecolors='white', s=120, marker='o')
            self.show_obs()
            self.camera.snap()
        return x, y


    def down(self, x, y):
        "Animate a downward move"
        for i in range(int(self.move_time * 10)):
            y += self.grid_size / (10 * self.move_time)
            self.ax.scatter(x, y, c='black', edgecolors='white', s=120, marker='o')
            self.show_obs()
            self.camera.snap()
        return x, y


    def show_obs(self):
        "Show all static objects: obstacles, boundaries, and goal"
        def adapt_coords(pos):
            row, col = pos
            return (col + 0.5, row + 0.5) if self.invert else (row + 0.5, col + 0.5)

        # Draw obstacles (red x)
        for pos in self.obs:
            x, y = adapt_coords(pos)
            self.ax.scatter(x, y, c='red', marker='x', s=100)

        # Draw boundaries (black x)
        for pos in self.bounds:
            x, y = adapt_coords(pos)
            self.ax.scatter(x, y, c='black', marker='x', s=100)

        # Draw goal (green star)
        gx, gy = adapt_coords(self.goal)
        self.ax.scatter(gx, gy, c='green', marker='*', s=200)

        # Draw legend once
        if not self._legend_drawn:
            self.ax.scatter([], [], c='red', marker='x', s=100, label='Obstacle')
            self.ax.scatter([], [], c='black', marker='x', s=100, label='Boundary')
            self.ax.scatter([], [], c='green', marker='*', s=200, label='Goal')
            self.ax.scatter([], [], c='black', edgecolors='white', s=120, marker='o', label='Agent')
            self._legend_drawn = True

    # Execute and animate a full trajectory from (cur_x, cur_y)
    def execute_traj(self, traj, cur_x, cur_y):
        # Optionally swap row and col based on display preference
        if self.invert:
            cur_x, cur_y = cur_y, cur_x

        # Offset to center agent in the middle of a grid cell
        x = cur_x + 0.5
        y = cur_y + 0.5

        # Move the agent step-by-step based on the trajectory list
        for move in traj:
            if move == "right":
                x, y = self.right(x, y)
            elif move == "left":
                x, y = self.left(x, y)
            elif move == "up":
                x, y = self.up(x, y)
            elif move == "down":
                x, y = self.down(x, y)
            else:
                # No move (could handle invalid move here)
                x, y = x, y

        # Show legend after the full trajectory is played
        self.ax.legend(loc='upper right', fontsize=10)



### Method for visualizing q-table with directions corresponding to optimal actions for each state ###

def draw_q_grid(q_table, env, scale=1.5, actions=["stay", "up", "down", "left", "right"], focus_center=None, focus_size=3):
    """
    Visualizes the Q-values of a grid-based environment using directional arrows and color-coded heatmaps.

    Args:
        q_table (dict): A dictionary with keys as (state, action) pairs and values as Q-values.
        env (GridWorld): The grid environment (used for size and boundary info).
        scale (float): Controls the visual scaling of the plot.
        actions (list): List of action names corresponding to Q-values.
        focus_center (tuple): Optional (x, y) center to zoom into a subsection of the grid.
        focus_size (int): Size of the subsection to show if using focus_center.
    """

    # === Determine region to visualize ===
    if focus_center:
        cx, cy = focus_center
        half = focus_size // 2
        row_range = range(max(0, cy - half), min(env.rows, cy + half + 1))
        col_range = range(max(0, cx - half), min(env.cols, cx + half + 1))
    else:
        row_range = range(env.rows)
        col_range = range(env.cols)

    # Arrow offsets for directional visualization
    dx, dy = 0.25, 0.25
    arrow_dict = {"U": (0, -dy), "D": (0, dy), "L": (-dx, 0), "R": (dx, 0)}

    # === Initialize plot ===
    fig, ax = plt.subplots(figsize=(len(col_range) * scale, len(row_range) * scale))
    ax.set_xlim(0, len(col_range))
    ax.set_ylim(0, len(row_range))
    ax.set_aspect('equal')
    ax.set_xticks(np.arange(0, len(col_range)+1, 1))
    ax.set_yticks(np.arange(0, len(row_range)+1, 1))
    ax.invert_yaxis()
    ax.grid(True)

    # === Normalize Q-values for color mapping ===
    all_q_vals = [v for (_, v) in q_table.items()]
    min_q = min(all_q_vals) if all_q_vals else -1
    max_q = max(all_q_vals) if all_q_vals else 1
    if min_q < 0 and max_q > 0:
        norm = TwoSlopeNorm(vmin=min_q, vcenter=0, vmax=max_q)
    else:
        norm = Normalize(vmin=min_q, vmax=max_q)
    cmap = plt.cm.bwr  # Blue-White-Red colormap

    # === Iterate over grid cells ===
    for i, row in enumerate(row_range):
        for j, col in enumerate(col_range):
            state = (row, col)

            # Get Q-values for each action at this state
            q_stay  = q_table.get((state, "stay"), 0)
            q_up    = q_table.get((state, "up"), 0)
            q_down  = q_table.get((state, "down"), 0)
            q_left  = q_table.get((state, "left"), 0)
            q_right = q_table.get((state, "right"), 0)

            # Cell center coordinates
            x, y = j, i
            cx, cy = x + 0.5, y + 0.5

            # Identify boundaries
            px, py = state
            boundary = px == 0 or px == env.rows - 1 or py == 0 or py == env.cols - 1

            # === Drawing helper functions ===
            def draw_triangle(points, q_val):
                color = 'black' if boundary else cmap(norm(q_val))
                triangle = patches.Polygon(points, closed=True, facecolor=color, edgecolor='black', alpha=0.85)
                ax.add_patch(triangle)

            def draw_square(origin):
                square = patches.Rectangle(origin, width=1, height=1, facecolor='black', edgecolor='black', alpha=1.0)
                ax.add_patch(square)

            def draw_stay_circle(center, q_val, radius=0.1):
                color = 'black' if (0 in state) or (env.cols-1 in state) else cmap(norm(q_val))
                circle = patches.Circle(center, radius, facecolor=color, edgecolor='black', alpha=0.9)
                ax.add_patch(circle)

            # === Render cell ===
            if boundary:
                draw_square((x, y))
            else:
                # Draw directional triangles for each action
                draw_triangle([(x, y), (x+1, y), (cx, cy)], q_up)
                draw_triangle([(x, y+1), (x+1, y+1), (cx, cy)], q_down)
                draw_triangle([(x, y), (x, y+1), (cx, cy)], q_left)
                draw_triangle([(x+1, y), (x+1, y+1), (cx, cy)], q_right)

                # Draw circle for 'stay' action
                draw_stay_circle((cx, cy), q_stay)

                # Determine best action and annotate with direction
                q_vals = [q_stay, q_up, q_down, q_left, q_right]
                best_action = actions[np.argmax(q_vals)]
                direction = best_action[0].upper()
                ax.text(cx, cy, direction, ha='center', va='center', fontsize=10, color='black')

                # Optional: draw directional arrow
                if direction in arrow_dict:
                    dx, dy = arrow_dict[direction]
                    ax.arrow(cx, cy, dx, dy, width=0.006)

    # === Add colorbar ===
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Required for colorbar to work
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label("Q-Value")

    # Final formatting
    plt.title("Q-Value Heatmap")
    plt.tight_layout()
    plt.show()

    ### Returns optimal action sequence given a policy ###

class Trajectory:
  """
  This class provides functionality for extracting or rolling
  out optimal trajectory from learned policy.

  Args:
      policy (dict): A dictionary with keys as state tuples and action strings as values.
      action_step (dict): A dictionary with keys as action strings and values as step tuples

  """

  def __init__(self,policy):
    self.policy = policy
    self.action_step = {
        "up":(-1,0),
        "down":(1,0),
        "right":(0,1),
        "left":(0,-1),
        "stay":(0,0)
    }



  def step(self, start_pos, goal_pos):
    "Recieves starting postion and goal position returns optimal action sequence"
    position = start_pos
    actions = []
    while not (position == goal):
      action = self.policy.get(position)
      actions.append(action)
      print(action)
      step = self.action_step.get(action)
      position = tuple(map(sum,zip(position, step)))
    return actions

#Setup MDP problem



### === Environment: GridWorld === ###
class GridWorld:
    """
    A 2D grid environment for reinforcement learning.
    Supports goal states, obstacles, boundaries, and rewards.
    """
    def __init__(self, obs: dict, goal: dict, rows: int, cols: int, bound_cost: int = -100, episode_steps: int = 1000):
        self.rows = rows
        self.cols = cols
        self.bound_cost = bound_cost
        self.episode_steps = episode_steps
        self.grid = np.full((rows, cols), 0)

        # Place obstacles with penalty values (must be > bound_cost)
        for (r, c), val in obs.items():
            if val <= bound_cost:
                raise ValueError(f"Obstacle value at ({r},{c}) is below or equal to bound_cost ({bound_cost}).")
            self.grid[r, c] = val

        # Place goals with positive rewards
        for (r, c), val in goal.items():
            self.grid[r, c] = val

        # Set outer boundaries
        self.grid[0, :] = bound_cost
        self.grid[-1, :] = bound_cost
        self.grid[:, 0] = bound_cost
        self.grid[:, -1] = bound_cost

        # Cache important locations
        self.goal_positions = list(zip(*np.where(self.grid > 0)))
        self.obstacle_positions = list(zip(*np.where((self.grid < -1) & (self.grid > bound_cost))))
        self.bound_positions = list(zip(*np.where(self.grid == bound_cost)))

    def get_reward(self, position):
        """Return the reward value at a given (row, col) position."""
        row, col = position
        return self.grid[row, col]

    def is_terminal(self, position):
        """Check if a state is a terminal (goal) state."""
        return position in self.goal_positions


### === Agent === ###
class Agent:
    """
    Grid-based agent that can move in 4 directions or stay in place.
    """
    def __init__(self, action_space=["stay", "up", "down", "left", "right"]):
        self.action_space = action_space

    def reset(self, start_pos):
        """Reset agent to a starting position."""
        self.position = start_pos

    def move(self, action, env: GridWorld):
        """
        Move the agent based on the chosen action, considering environment boundaries.
        """
        row, col = self.position
        if action == "up" and row > 0:
            row -= 1
        elif action == "down" and row < env.rows - 1:
            row += 1
        elif action == "left" and col > 0:
            col -= 1
        elif action == "right" and col < env.cols - 1:
            col += 1
        # "stay" does nothing
        self.position = (row, col)
        return self.position

### === Q-Learning Algorithm === ###
class QLearning:
    """
    Tabular Q-learning agent for discrete state-action environments.
    """
    def __init__(self, env: GridWorld, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.alpha = alpha                  # Learning rate
        self.gamma = gamma                  # Discount factor
        self.epsilon = epsilon              # Exploration rate
        self.q_table = {}                   # Stores Q-values: (state, action) -> float
        self.actions = ["stay", "up", "down", "left", "right"]
        self.env = env

    def get_q(self, state, action):
        """Return Q-value for a given state-action pair; default to 0.0."""
        return self.q_table.get((state, action), 0.0)

    def choose_action(self, state):
        """
        Choose an action using epsilon-greedy strategy.
        With probability epsilon, explore; otherwise exploit.
        """
        if random.random() < self.epsilon:
            return random.choice(self.actions)
        q_vals = [self.get_q(state, a) for a in self.actions]
        return self.actions[np.argmax(q_vals)]

    def update(self, state, action, reward, next_state):
        """
        Perform the Q-learning update:
        Q(s, a) ← Q(s, a) + α [r + γ max_a' Q(s', a') − Q(s, a)]
        """

        max_q_next = max([self.get_q(next_state, a) for a in self.actions])
        old_q = self.get_q(state, action)
        new_q = old_q + self.alpha * (reward + self.gamma * max_q_next - old_q)
        self.q_table[(state, action)] = new_q

def recover_policy(q_table, actions, rows, cols):
    """
    Recover the best policy (mapping of state → optimal action) from a Q-table.

    Args:
        q_table (dict): A dictionary mapping (state, action) tuples to Q-values.
        actions (list): List of all possible actions (strings).
        rows (int): Number of rows in the grid environment.
        cols (int): Number of columns in the grid environment.

    Returns:
        dict: A policy mapping each state (tuple) to its best action.
    """
    shape = (rows, cols)
    states = []

    # === Generate and print all states ===
    print("### States ###")
    for row in range(rows):
        part_states = [(row, col) for col in range(shape[1])]
        print(*part_states)
        states += part_states

    # === Show available actions ===
    print("\n### Actions ###")
    print(actions)

    policy = {}

    # === Determine best action for each state based on Q-values ===
    for state in states:
        # Get Q-values for each possible action from this state
        q_values = [q_table.get((state, a), 0) for a in actions]

        # Choose action with highest Q-value
        best_action_idx = np.argmax(q_values)
        policy[state] = actions[best_action_idx]

    return policy


def train(agent, env, qlearn, episodes=1000, random_init: bool = False):
    """
    Train an agent in a GridWorld environment using Q-learning.

    Args:
        agent (Agent): The agent that moves through the environment.
        env (GridWorld): The grid-based environment.
        qlearn (QLearning): The Q-learning algorithm instance.
        episodes (int): Number of episodes to run.
        random_init (bool): If True, agent starts at a random position each episode.
    """

    for ep in range(episodes):
        # === Initialize Agent ===
        if not random_init:
            agent.reset((0, 0))  # Default starting point
        else:
            # Start randomly within inner grid (avoid boundary walls)
            agent.reset((random.randint(1, env.rows - 1), random.randint(1, env.cols - 1)))

        state = agent.position  # Initial state

        for _ in range(env.episode_steps):
            # === SAMPLE ACTION ===
            action = qlearn.choose_action(state)

            # === INTERACT WITH ENVIRONMENT ===
            next_state = agent.move(action, env)

            # === EVALUATE REWARD ===
            # Note: Reward is based on the result of the transition (s, a → s')
            # We care about the consequence of the action, not just being in a state
            reward = env.get_reward(next_state)

            # === LEARNING / Q-VALUE UPDATE ===
            qlearn.update(state, action, reward, next_state)

            # === PROCEED TO NEXT STATE ===
            state = next_state




Note: you may need to restart the kernel to use updated packages.


##TODO: Initialize and Train our Policy

Use our Q-Learning implementation to solve a representation of the maze we've set up in the lab!

In [ ]:
rows, cols = ...

obs = ...

goal = ...

env = ...

agent = ...

qlearn = ...

print("Grid Layout")
print(env.grid)

In [ ]:
draw_q_grid(qlearn.q_table, env, scale=1)


In [ ]:
# TODO: Write the call to train our qlearn algorithm on the environment we've set up

# Show the learned value grid
draw_q_grid(qlearn.q_table, env, scale=1)

# Recover policy
policy = recover_policy(qlearn.q_table,qlearn.actions,rows,cols)

In [ ]:
#main - test animation -- all good!
scale = 1
fig, ax = plt.subplots(figsize=(env.cols * scale, env.rows * scale))

camera = Camera(fig)# the camera gets our figure


ax.set_xlim(0, env.cols)
ax.set_ylim(0, env.rows)
ax.set_aspect('equal')
ax.set_xticks(np.arange(0, env.cols+1, 1))
ax.set_yticks(np.arange(0, env.rows+1, 1))
ax.invert_yaxis()
ax.grid(True)



obs = env.obstacle_positions
bounds = env.bound_positions
goal = env.goal_positions[0]
anim = Move_anim(ax, camera, obs, goal, bounds, invert=True)


start = (2,8)
trajectories = Trajectory(policy=policy)
traj = trajectories.step(start,goal)

anim.execute_traj(traj, *start)


animation = camera.animate()
plt.close()
HTML(animation.to_html5_video())

###Controlling the Drone to Solve the Maze:





In [ ]:
#Write the Actual Code to run the drone here
# example!

rows, cols = 5, 5
# Define obstacle and goal positions
obs = {
    (1, 2): -40,
    (3, 3): -20
}
goal = {
    (3, 1): 100
}
env = GridWorld(obs=obs, goal=goal, rows=rows, cols=cols, episode_steps=100)
agent = Agent()
qlearn = QLearning(env=env,epsilon=0.8)

train(agent,env,qlearn, 100, random_init=True)
print("train finished")
policy = recover_policy(qlearn.q_table,qlearn.actions,rows,cols)

goal = env.goal_positions[0]

start = (1,3)
trajectories = Trajectory(policy=policy)
traj = trajectories.step(start,goal)

In [ ]:
drone = Drone_Controller(1, 5, 5, group_number)

initial = np.array([0,0])

seq = drone.convert_traj_to_setpoint(policy, (1,4), (3,1), 5, (0,0))

drone.execute_sequence(seq)